In [1]:
export_folder = "seg1"
root_folder = r"j:\Temp"

import os

export_fullpath = os.path.join(root_folder, export_folder)

if not os.path.exists(export_fullpath):
    os.makedirs(export_fullpath)
    print("Created folder: " + export_fullpath)

print "Export data to: " + export_fullpath

Export data to: j:\Temp\seg1


In [2]:
labelmap_name = "Segmentation-label"
image_name = "-Image"

labelmap_node = slicer.util.getFirstNodeByName(labelmap_name)
labelmap_image = labelmap_node.GetImageData()
print "Labelmap dimensions: " + str(labelmap_image.GetDimensions())

image_node = slicer.util.getFirstNodeByName(image_name, className="vtkMRMLScalarVolumeNode")
print "Ultrasound image node: " + image_node.GetID()

sequence_browser_node = slicer.util.getFirstNodeByName('', className='vtkMRMLSequenceBrowserNode')
print "Sequence browser node: " + str(sequence_browser_node.GetID())

image_transform_node = slicer.util.getNode(image_node.GetTransformNodeID())
# image_to_reference_matrix = vtk.vtkMatrix4x4()
# image_transform_node.GetMatrixTransformToWorld(image_to_reference_matrix)
print "ImageToReference transform node: " + image_transform_node.GetName()

Labelmap dimensions: (344, 103, 106)
Ultrasound image node: vtkMRMLScalarVolumeNode1
Sequence browser node: vtkMRMLSequenceBrowserNode1
ImageToReference transform node: SkullScan20160719_110911_ImageToReference-ImageToReference


In [3]:
image_to_reference_matrix = vtk.vtkMatrix4x4()
image_transform_node.GetMatrixTransformToWorld(image_to_reference_matrix)
image_to_reference_transform = vtk.vtkTransform()
image_to_reference_transform.SetMatrix(image_to_reference_matrix)

t = image_to_reference_transform.GetPosition()
o = image_to_reference_transform.GetOrientationWXYZ()

reslice_to_reference_transform = vtk.vtkTransform()
reslice_to_reference_transform.Translate(t)
reslice_to_reference_transform.RotateWXYZ(o[0], o[1], o[2], o[3])

reslice_to_reference_matrix = vtk.vtkMatrix4x4()
reslice_to_reference_transform.GetMatrix(reslice_to_reference_matrix)

In [4]:
reslice = vtk.vtkImageReslice()
reslice.SetInputData(labelmap_image)
reslice.SetOutputDimensionality(2)
reslice.SetInterpolationModeToLinear()
reslice.SetResliceAxes(reslice_to_reference_matrix)
reslice.Update()
labelmap_reslice_image = reslice.GetOutput()

dims = labelmap_reslice_image.GetDimensions()
print "Resliced image dimensions: " + str(dims)

Resliced image dimensions: (331, 118, 1)


In [5]:
png_file_name = "vtk_png_writer.png"
png_fullpath = os.path.join(export_fullpath, png_file_name)

png_writer = vtk.vtkPNGWriter()
png_writer.SetInputData(labelmap_reslice_image)
png_writer.SetFileName(png_fullpath)
png_writer.Update()
png_writer.Write()

print "Image exported to: " + png_fullpath

Image exported to: j:\Temp\seg1\vtk_png_writer.png


In [6]:
from vtk.util import numpy_support
import numpy as np

dims = labelmap_reslice_image.GetDimensions()

a = numpy_support.vtk_to_numpy(labelmap_reslice_image.GetPointData().GetScalars())
# b = a.reshape(dims[0], dims[1], dims[2])
b = a.reshape(dims[0], dims[1])

print "dims: " + str(dims)
print str(b.shape)
print "max: " + str(a.max())
print "min: " + str(a.min())

arr_fullpath = os.path.join(export_fullpath, "numpy_array")
np.save(arr_fullpath, b)

print "Writing numpy array to: " + arr_fullpath

dims: (331, 118, 1)
(331L, 118L)
max: 1
min: 0
Writing numpy array to: j:\Temp\seg1\numpy_array


In [7]:
import cv2

jpg_fullpath = os.path.join(export_fullpath, "OpenCV_imwrite.jpg")

bc = np.zeros([dims[0], dims[1], 3])

print str(bc[0][:,:].shape)
print str(b[:,:].shape)

bc[:,:,0] = b[:,:]
bc[:,:,1] = b[:,:]
bc[:,:,2] = b[:,:]

print str(bc.shape)

cv2.imwrite(jpg_fullpath, bc)

(118L, 3L)
(331L, 118L)
(331L, 118L, 3L)
